In [2]:

# To examine the geographical distribution of job openings
# Import Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
from us import states

# Google developer API key
from config import gkey

In [3]:
# Read CSV 
data_df = pd.read_csv('glassdoordata.csv')

renamed_data_df = data_df.rename(columns={"City, State":"city_state"})
renamed_data_df.head()

,Company Name Witheld,Job Title,Pay Type,High Salary,Low Salry,Job Description,Country,city_state,Industry,Rating Average,Number of Ratings
0,"Auris Health, Inc.",Program Manager,NaN,NaN,NaN,<strong>Overview:</strong>\n<br/><br/>\nAs the...,US,"Redwood City, CA",Healthcare Product Manufacturing,0.75,2812
1,Applications Software Technology LLC,Associate Consultant- Technology,ANNUAL,76591.0,52158.0,<strong>Have a Technology Background?\n<br/>\n...,US,"Lisle, IL",IT Services,0.99,2850
2,Seen by Indeed,Project Manager,NaN,NaN,NaN,<div><div><div><strong>Seen by Indeed is a fre...,US,"San Francisco, CA",NaN,NaN,2869
3,Power Monitors,Sales Engineer,ANNUAL,71791.0,47259.0,"<p>Power Monitors, Inc. is seeking a Sales Eng...",US,"Mount Crawford, VA",Electrical & Electronic Manufacturing,0.31,2875
4,Carnegie Robotics,Senior Mechanical Engineer,ANNUAL,162918.0,64786.0,<p><strong>Role:</strong> Senior Mechanical En...,US,"Pittsburgh, PA",Electrical & Electronic Manufacturing,1.00,2931


In [4]:

# grouped_citystate = DSjob_Feb2019.groupby('city_state')['category'].count().sort_values(ascending=False)
grouped_citystate = renamed_data_df.groupby('city_state')['Industry'].count().sort_values(ascending=False).reset_index()
grouped_citystate.head(300)

,city_state,Industry
0,"San Francisco, CA",157
1,"New York, NY",118
2,"Boston, MA",37
3,"Chicago, IL",33
4,"Denver, CO",31
...,...,...
295,"White Plains, NY",1
296,"Billerica, MA",1
297,"Arlington, TN",1
298,"Arden Hills, MN",1


In [5]:

grouped_citystate['lat'] = 999
grouped_citystate['lng']= 999
grouped_citystate

,city_state,Industry,lat,lng
0,"San Francisco, CA",157,999,999
1,"New York, NY",118,999,999
2,"Boston, MA",37,999,999
3,"Chicago, IL",33,999,999
4,"Denver, CO",31,999,999
...,...,...,...,...
439,"Chagrin Falls, OH",0,999,999
440,"Greenville, SC",0,999,999
441,"Champaign, IL",0,999,999
442,"Greensboro, NC",0,999,999


In [6]:
for irow in np.arange(len(grouped_citystate)):
    print(irow)
    target_city = grouped_citystate.city_state[irow]
    # Build the endpoint URL
    target_url = f'https://maps.googleapis.com/maps/api/geocode/json?address={target_city}&key={gkey}'
    
    # Run a request to endpoint and convert result to json
    geo_data = requests.get(target_url).json()
#     print(json.dumps(geo_data, indent=4, sort_keys=True))
    # Extract latitude and longitude
    try:
        grouped_citystate.loc[irow, 'lat'] = geo_data["results"][0]["geometry"]["location"]["lat"]
        grouped_citystate.loc[irow, 'lng'] = geo_data["results"][0]["geometry"]["location"]["lng"]
    except:
        grouped_citystate.loc[irow, 'lat'] = '999'
        grouped_citystate.loc[irow, 'lng'] = '999'

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [7]:
grouped_citystate

,city_state,Industry,lat,lng
0,"San Francisco, CA",157,37.774929,-122.419415
1,"New York, NY",118,40.712775,-74.005973
2,"Boston, MA",37,42.360082,-71.058880
3,"Chicago, IL",33,41.878114,-87.629798
4,"Denver, CO",31,39.739236,-104.990251
...,...,...,...,...
439,"Chagrin Falls, OH",0,41.429852,-81.391100
440,"Greenville, SC",0,34.852618,-82.394010
441,"Champaign, IL",0,40.116420,-88.243383
442,"Greensboro, NC",0,36.072635,-79.791975


In [8]:

# Create a job opening Heatmap layer
locations = grouped_citystate[["lat", "lng"]].astype(float)
locations

,lat,lng
0,37.774929,-122.419415
1,40.712775,-74.005973
2,42.360082,-71.058880
3,41.878114,-87.629798
4,39.739236,-104.990251
...,...,...
439,41.429852,-81.391100
440,34.852618,-82.394010
441,40.116420,-88.243383
442,36.072635,-79.791975


In [9]:
grouped_citystate.Industry

0      157
1      118
2       37
3       33
4       31
      ... 
439      0
440      0
441      0
442      0
443      0
Name: Industry, Length: 444, dtype: int64

In [14]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=grouped_citystate.Industry, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))